In [1]:
import time
import random
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
from collections import Counter

import utils

In [2]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import zipfile

dataset_folder_path = 'data'
dataset_filename = 'text8.zip'
dataset_name = 'Text8 Dataset'

class DLProgess(tqdm):
    last_block = 0
    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num
        
if not isfile(dataset_filename):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc=dataset_name) as pbar:
        urlretrieve(
                'http://mattmahoney.net/dc/text8.zip',
                dataset_filename,
                pbar.hook)
        
if not isdir(dataset_folder_path):
    with zipfile.ZipFile(dataset_filename) as zip_ref:
        zip_ref.extractall(dataset_folder_path)
        
with open('data/text8') as f:
    text = f.read()

In [3]:
def proprocess(text):
    
    text = text.lower()
    text = text.replace('.', ' <PREIOD> ')
    text = text.replace(',', ' <COMMA> ')
    text = text.replace('"', ' <QUOTATION_MARK> ')
    text = text.replace(';', ' <SEMICOLON> ')
    text = text.replace('!', ' <EXCLAMATION_MARK> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    text = text.replace('(', ' <LEFT_PAREN> ')
    text = text.replace(')', ' <RIGHT-PAREN> ')
    text = text.replace('--', '< HYPHENS >')
    text = text.replace('?', '< QUESTION_MARK >')
    words = text.replace(':', ' <COLON> ')
    words = text.split()
    
    word_counts = Counter(words)
    trimmed_words = [word for word in words if word_counts[word] > 5]
    
    return trimmed_words

words = utils.preprocess(text)
print (words[:30])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst']


In [4]:
print ("Total words: {}".format(len(words)))
print ("Unique words: {}".format(len(set(words))))

Total words: 16680599
Unique words: 63641


In [5]:
def create_lookup_tables(words):
    word_counts = Counter(words)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}
    
    return vocab_to_int, int_to_vocab

In [6]:
vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

In [7]:
t = 1e-5
count_words = Counter(int_words)
total_words = len(int_words)
freq_words = {word: count_word/total_words for word, count_word in count_words.items()}
p_drop = {word: 1-np.sqrt(t/freq) for word, freq in freq_words.items()}

train_words = [word for word, p in p_drop.items() if random.random() < p]

In [8]:
def get_target(words, idx, window_size=5):
#     sub_win = random.randint(1, window_size+1)
    sub_win = window_size//2
    start = idx - sub_win if (idx - sub_win) > 0 else 0
    end = idx + sub_win
    target_words = set(words[start: idx] + words[idx+1: end+1])
    
    return list(target_words)

In [14]:
def get_batches(words, batch_size, window_size=5):
    n_batches = len(words) // batch_size
    
    # only full batches
    words = words[: n_batches*batch_size]
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx: idx+batch_size]
        for ii in range(window_size//2, len(batch)-window_size//2):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x])
        
            yield batch_x, batch_y
#         yield x, y

In [10]:
class Net(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, windows_size, n_hidden):
        super(Net, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.hidden = nn.Linear(embedding_dim, n_hidden)
        self.output = nn.Linear(n_hidden, (windows_size-1)*vocab_size)
        self.windows_size = windows_size
        
    def forward(self, x):
        emb = self.embeddings(x)
        h = self.hidden(emb)
        h = F.relu(h)
        out = self.output(h)
        log_prob = F.log_softmax(out, dim=1).view(self.windows_size-1, -1)
        
        return log_prob



In [13]:
epoches = 100
window_size = 5
hidden = 128
vocab_size = len(vocab_to_int)
embedding_dim = 100

batch_size = 10
net = Net(vocab_size, embedding_dim, window_size, hidden)
criterion = nn.NLLLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=1e-3)
net = net.to('cuda')
for epoch in range(epoches):
    running_loss = 0
    
    batches = get_batches(train_words, batch_size, window_size)
    for x, y in batches:
#         print (x, y)
        x, y = torch.LongTensor([x]).to('cuda'), torch.LongTensor(y).to('cuda')
#         print (x)
        out = net(x)
#         print (out, y)
        loss = criterion(out, y)

        running_loss += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print ('Loss: {:.6f}'.format(running_loss/ len(vocab_to_int)))

ValueError: Expected input batch_size (4) to match target batch_size (24).

In [149]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
# print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:
        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
#         print (context_idxs)
        
        log_probs = model(context_idxs)
        
        print (log_probs)
        print (torch.tensor([word_to_ix[target]], dtype=torch.long))
        break
        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    break
    losses.append(total_loss)
    
print(losses)  # The loss decreased every iteration over the training data!

tensor([[-4.3538, -4.6007, -5.0398, -4.9648, -4.6309, -4.4803, -4.7688, -4.9358,
         -4.1981, -4.9230, -4.2645, -4.2247, -4.5440, -3.8349, -5.1897, -4.6669,
         -4.2760, -4.6739, -5.0728, -4.5043, -4.7013, -4.3932, -4.4034, -3.9276,
         -5.0377, -5.1229, -4.6699, -4.8776, -5.1106, -3.9835, -4.6497, -4.8369,
         -5.4839, -4.3270, -4.1150, -4.9706, -4.5220, -4.6367, -4.2472, -5.1215,
         -4.6386, -5.0091, -5.0965, -4.5019, -4.8320, -4.1742, -4.5657, -5.1686,
         -4.3551, -4.2830, -4.8977, -4.5800, -4.8081, -5.1652, -5.0385, -4.4137,
         -4.5179, -4.9374, -3.9648, -4.3845, -4.1027, -4.4440, -4.0493, -4.3854,
         -4.8065, -4.5743, -4.5261, -5.0209, -4.9812, -5.0853, -4.1536, -5.3433,
         -3.8726, -4.8139, -4.7930, -4.7320, -4.7073, -4.1928, -4.5084, -4.0860,
         -4.7967, -5.0510, -4.6348, -5.1435, -4.7554, -4.5807, -4.9860, -4.5273,
         -4.9976, -4.4065, -4.5494, -4.6530, -5.0937, -4.4996, -4.6472, -4.9710,
         -4.0610]], grad_fn=

In [173]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import urllib.request
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize
import sklearn
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances

torch.manual_seed(1)

CONTEXT_SIZE = 3
EMBEDDING_DIM = 10

def get_key(word_id):
    for key,val in word_to_ix.items():
        if(val == word_id):
            print(key)


def cluster_embeddings(filename,nclusters):
    X = np.load(filename)
    kmeans = KMeans(n_clusters=nclusters, random_state=0).fit(X)
    center = kmeans.cluster_centers_
    distances = euclidean_distances(X,center)

    for i in np.arange(0,distances.shape[1]):
        word_id = np.argmin(distances[:,i])
        print(word_id)
        get_key(word_id)

def read_data(file_path):
    tokenizer = RegexpTokenizer(r'\w+')
    data = urllib.request.urlopen(file_path)
    data = data.read().decode('utf8')
    tokenized_data = word_tokenize(data)
    stop_words = set(stopwords.words('english'))
    stop_words.update(['.',',',':',';','(',')','#','--','...','"'])
    cleaned_words = [ i for i in tokenized_data if i not in stop_words ]
    return(cleaned_words)


test_sentence = """Empathy for the poor may not come easily to people who never experienced it. They may blame the victims and insist their predicament can be overcome through determination and hard work.
But they may not realize that extreme poverty can be psychologically and physically incapacitating — a perpetual cycle of bad diets, health care and education exacerbated by the shaming and self-fulfilling prophecies that define it in the public imagination.
Gordon Parks — perhaps more than any artist — saw poverty as “the most savage of all human afflictions” and realized the power of empathy to help us understand it. It was neither an abstract problem nor political symbol, but something he endured growing up destitute in rural Kansas and having spent years documenting poverty throughout the world, including the United States.
That sensitivity informed “Freedom’s Fearful Foe: Poverty,” his celebrated photo essay published in Life magazine in June 1961. He took readers into the lives of a Brazilian boy, Flavio da Silva, and his family, who lived in the ramshackle Catacumba favela in the hills outside Rio de Janeiro. These stark photographs are the subject of a new book, “Gordon Parks: The Flavio Story” (Steidl/The Gordon Parks Foundation), which accompanies a traveling exhibition co-organized by the Ryerson Image Centre in Toronto, where it opens this week, and the J. Paul Getty Museum. Edited with texts by the exhibition’s co-curators, Paul Roth and Amanda Maddox, the book also includes a recent interview with Mr. da Silva and essays by Beatriz Jaguaribe, Maria Alice Rezende de Carvalho and Sérgio Burgi.
""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)

#test_sentence = read_data('https://www.gutenberg.org/files/57884/57884-0.txt')

ngrams = []
for i in range(len(test_sentence) - CONTEXT_SIZE):
    tup = [test_sentence[j] for j in np.arange(i + 1 , i + CONTEXT_SIZE + 1) ]
    ngrams.append((test_sentence[i],tup))
# print the first 3, just so you can see what they look like
#print(ngrams)


vocab = set(test_sentence)
print("Length of vocabulary",len(vocab))
word_to_ix = {word: i for i, word in enumerate(vocab)}


class SkipgramModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(SkipgramModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.linear2 = nn.Linear(128, context_size * vocab_size)
        #self.parameters['context_size'] = context_size

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))  # -1 implies size inferred for that index from the size of the data
        #print(np.mean(np.mean(self.linear2.weight.data.numpy())))
        out1 = F.relu(self.linear1(embeds)) # output of first layer
        out2 = self.linear2(out1)           # output of second layer
        #print(embeds)
        log_probs = F.log_softmax(out2, dim=1).view(CONTEXT_SIZE,-1)
        return log_probs

    def predict(self,input):
        context_idxs = torch.tensor([word_to_ix[input]], dtype=torch.long)
        res = self.forward(context_idxs)
        res_arg = torch.argmax(res)
        res_val, res_ind = res.sort(descending=True)
        indices = [res_ind[i][0] for i in np.arange(0,3)]
        for arg in indices:
            print( [ (key, val) for key,val in word_to_ix.items() if val == arg ])


    def freeze_layer(self,layer):
        for name,child in model.named_children():
            print(name,child)
            if(name == layer):
                for names,params in child.named_parameters():
                    print(names,params)
                    print(params.size())
                    params.requires_grad= False

    def print_layer_parameters(self):
        for name,child in model.named_children():
                print(name,child)
                for names,params in child.named_parameters():
                    print(names,params)
                    print(params.size())

    def write_embedding_to_file(self,filename):
        for i in self.embeddings.parameters():
            weights = i.data.numpy()
        np.save(filename,weights)


losses = []
loss_function = nn.NLLLoss()
model = SkipgramModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Freeze embedding layer
#model.freeze_layer('embeddings')

for epoch in range(550):
    total_loss = 0
    #------- Embedding layers are trained as well here ----#
    #lookup_tensor = torch.tensor([word_to_ix["poor"]], dtype=torch.long)
    #hello_embed = model.embeddings(lookup_tensor)
    #print(hello_embed)
    # -----------------------------------------------------#

    model.predict('psychologically')

    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        #print(context,target)


        context_idxs = torch.tensor([word_to_ix[context]], dtype=torch.long)
        #print("Context id",context_idxs)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)
        #print(log_probs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        target_list = torch.tensor([word_to_ix[w] for w in target], dtype=torch.long)
        loss = loss_function(log_probs, target_list)
        #print(loss)

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    print(total_loss)
    losses.append(total_loss)
#print(losses)  # The loss decreased every iteration over the training data!

#Print the model layer parameters
#model.print_layer_parameters()

#Predict the next word given n context words
model.predict('psychologically')
model.write_embedding_to_file('embeddings_skipgrams.npy')
cluster_embeddings('embeddings_skipgrams.npy',5)

Length of vocabulary 195
[('insist', 77)]
[('afflictions”', 83)]
[('Alice', 88)]
1644.6528630256653
[('insist', 77)]
[('afflictions”', 83)]
[('Alice', 88)]
1642.0468792915344
[('victims', 160)]
[('afflictions”', 83)]
[('Alice', 88)]
1639.4606614112854
[('victims', 160)]
[('afflictions”', 83)]
[('Alice', 88)]
1636.8939280509949
[('victims', 160)]
[('afflictions”', 83)]
[('Alice', 88)]
1634.3458337783813
[('victims', 160)]
[('afflictions”', 83)]
[('Alice', 88)]
1631.8149962425232
[('victims', 160)]
[('afflictions”', 83)]
[('Alice', 88)]
1629.3006920814514
[('victims', 160)]
[('afflictions”', 83)]
[('Alice', 88)]
1626.8015265464783
[('victims', 160)]
[('afflictions”', 83)]
[('Alice', 88)]
1624.3170342445374
[('victims', 160)]
[('the', 136)]
[('Alice', 88)]
1621.8456707000732
[('victims', 160)]
[('the', 136)]
[('Alice', 88)]
1619.3868370056152
[('victims', 160)]
[('the', 136)]
[('Alice', 88)]
1616.9397797584534
[('victims', 160)]
[('the', 136)]
[('Alice', 88)]
1614.5040121078491
[('victims

[('in', 31)]
997.0416456460953
[('and', 139)]
[('can', 113)]
[('in', 31)]
994.017196059227
[('and', 139)]
[('can', 113)]
[('in', 31)]
990.9975258111954
[('and', 139)]
[('can', 113)]
[('in', 31)]
987.9816936254501
[('and', 139)]
[('can', 113)]
[('in', 31)]
984.9692245721817
[('and', 139)]
[('can', 113)]
[('in', 31)]
981.9608970880508
[('and', 139)]
[('can', 113)]
[('in', 31)]
978.9571114778519
[('and', 139)]
[('can', 113)]
[('in', 31)]
975.9581354856491
[('and', 139)]
[('can', 113)]
[('in', 31)]
972.9644321203232
[('and', 139)]
[('can', 113)]
[('in', 31)]
969.9748946428299
[('and', 139)]
[('can', 113)]
[('in', 31)]
966.9911109209061
[('and', 139)]
[('can', 113)]
[('in', 31)]
964.0126566886902
[('and', 139)]
[('can', 113)]
[('in', 31)]
961.0398725271225
[('and', 139)]
[('can', 113)]
[('in', 31)]
958.0724025964737
[('and', 139)]
[('can', 113)]
[('in', 31)]
955.1109210252762
[('and', 139)]
[('can', 113)]
[('in', 31)]
952.1566935777664
[('and', 139)]
[('can', 113)]
[('in', 31)]
949.20753347

KeyboardInterrupt: 